# BLM example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Add pytwoway to system path, do not run this
import sys
sys.path.append('../../..')

In [3]:
# Import the pytwoway package 
# (Make sure you have installed it using pip install pytwoway)
import pytwoway as tw
import pandas as pd

## Simulate some data

The package contains functions to simulate data. We use this here to keep things simple.

In [6]:
# For the example, we simulate data
sim_model = tw.BLMModel({
    'nl': 6, # Number of worker types
    'nk': 10, # Number of firm types
    'simulation': True # Simulating data
})
sim_data = sim_model._m2_mixt_simulate_sim()
display(sim_data['jdata'])
display(sim_data['sdata'])

,l,y1,y2,j1,j2,j1true,j2true,f1,f2
0,5,0.624487,-0.295308,0,0,0,0,F3,F1
1,3,-0.056883,1.019265,0,0,0,0,F4,F3
2,5,-0.017014,-1.122528,0,0,0,0,F3,F7
3,5,-0.623453,-1.280983,0,0,0,0,F9,F0
4,0,0.055882,0.426383,0,0,0,0,F10,F3
...,...,...,...,...,...,...,...,...,...
995,0,1.818979,2.256241,9,9,9,9,F2,F0
996,5,2.015726,1.419215,9,9,9,9,F6,F7
997,5,2.186478,1.331281,9,9,9,9,F9,F6
998,0,1.425481,2.189457,9,9,9,9,F2,F10


,k,y1,y2,j1,j2,x,f1,j1true,f2
0,5,-0.110349,-0.538967,0,0,1,F0,0,F0
1,0,0.324518,0.543298,0,0,1,F0,0,F0
2,3,0.091016,0.906354,0,0,1,F0,0,F0
3,3,0.167321,0.675426,0,0,1,F0,0,F0
4,0,0.899660,0.631845,0,0,1,F0,0,F0
...,...,...,...,...,...,...,...,...,...
95,4,1.374347,1.281357,9,9,1,F10,9,F10
96,5,1.728342,1.447324,9,9,1,F10,9,F10
97,4,2.215635,1.415000,9,9,1,F10,9,F10
98,4,1.722898,1.210873,9,9,1,F10,9,F10


# Initialize and run BLMEstimator

In [7]:
## Optional Parameters ##
fit_params = {
    # Class parameters
    'nl': 6, # Number of worker types
    'nk': 10, # Number of firm types
    'fixb': False, # Set A2 = np.mean(A2, axis=0) + A1 - np.mean(A1, axis=0)
    'stationary': False, # Set A1 = A2
    'simulation': False, # If True, using model to simulate data
    # fit_movers() and fit_stayers() parameters
    'maxiters': 100, # Max number of iterations
    # fit_movers() parameters
    'update_a': True, # If False, do not update A1 or A2
    'update_s': True, # If False, do not update S1 or S2
    'return_qi': False, # If True, return qi matrix after first loop
    'cons_a': (['lin'], {'n_periods': 2}), # Constraints on A1 and A2
    'cons_s': (['biggerthan'], {'gap_bigger': 1e-4, 'n_periods': 2}), # Constraints on S1 and S2
    # fit_stayers() parameters
    'return_qi': False # If True, return qi matrix after first loop
}
# Initialize BLM estimator
BLM = tw.BLMEstimator()
# Fit BLM estimator
BLM.fit(jdata=sim_data['jdata'], sdata=sim_data['sdata'], iter=10, ncore=1, user_params=fit_params)

Running fixm movers
loop 0, liks 15.667380552307195
loop 1, liks 18.611198529111114
loop 2, liks 18.637894672119913
loop 3, liks 18.638977637902205
loop 4, liks 18.63904297577522
loop 5, liks 18.639048049338697
loop 6, liks 18.639048519790972
loop 7, liks 18.639048567833083
loop 8, liks 18.639048572972214
loop 9, liks 18.639048573534026
loop 10, liks 18.63904857359608
loop 11, liks 18.639048573602963
loop 12, liks 18.639048573603734
loop 13, liks 18.639048573603816
loop 14, liks 18.639048573603826
loop 15, liks 18.639048573603826
loop 16, liks 18.639048573603826
loop 17, liks 18.63904857360383
loop 18, liks 18.63904857360383
loop 19, liks 18.639048573603826
loop 20, liks 18.63904857360383
loop 21, liks 18.63904857360383
loop 22, liks 18.63904857360383
loop 23, liks 18.639048573603826
loop 24, liks 18.63904857360383
loop 25, liks 18.63904857360383
loop 26, liks 18.63904857360383
loop 27, liks 18.639048573603826
loop 28, liks 18.639048573603826
loop 29, liks 18.63904857360383
loop 30, li

loop 51, liks 18.639048573603826
loop 52, liks 18.639048573603826
loop 53, liks 18.639048573603826
loop 54, liks 18.639048573603826
loop 55, liks 18.639048573603826
loop 56, liks 18.639048573603826
loop 57, liks 18.639048573603826
loop 58, liks 18.639048573603826
loop 59, liks 18.639048573603826
loop 60, liks 18.639048573603826
loop 61, liks 18.639048573603826
loop 62, liks 18.639048573603826
loop 63, liks 18.639048573603826
loop 64, liks 18.639048573603826
loop 65, liks 18.639048573603826
loop 66, liks 18.639048573603826
loop 67, liks 18.639048573603826
loop 68, liks 18.639048573603826
loop 69, liks 18.639048573603826
loop 70, liks 18.639048573603826
loop 71, liks 18.639048573603826
loop 72, liks 18.639048573603826
loop 73, liks 18.639048573603826
loop 74, liks 18.639048573603826
loop 75, liks 18.639048573603826
loop 76, liks 18.639048573603826
loop 77, liks 18.639048573603826
loop 78, liks 18.639048573603826
loop 79, liks 18.639048573603826
loop 80, liks 18.639048573603826
loop 81, l

loop 15, liks 18.63904857360383
loop 16, liks 18.639048573603826
loop 17, liks 18.639048573603823
loop 18, liks 18.63904857360383
loop 19, liks 18.639048573603826
loop 20, liks 18.63904857360383
loop 21, liks 18.63904857360383
loop 22, liks 18.639048573603826
loop 23, liks 18.639048573603823
loop 24, liks 18.639048573603826
loop 25, liks 18.63904857360383
loop 26, liks 18.63904857360383
loop 27, liks 18.63904857360383
loop 28, liks 18.63904857360383
loop 29, liks 18.63904857360383
loop 30, liks 18.63904857360383
loop 31, liks 18.639048573603826
loop 32, liks 18.639048573603826
loop 33, liks 18.639048573603826
loop 34, liks 18.639048573603826
loop 35, liks 18.639048573603826
loop 36, liks 18.639048573603826
loop 37, liks 18.639048573603826
loop 38, liks 18.639048573603826
loop 39, liks 18.639048573603826
loop 40, liks 18.639048573603826
loop 41, liks 18.639048573603826
loop 42, liks 18.639048573603826
loop 43, liks 18.639048573603826
loop 44, liks 18.639048573603826
loop 45, liks 18.639

loop 77, liks 18.639048573603826
loop 78, liks 18.639048573603826
loop 79, liks 18.639048573603826
loop 80, liks 18.639048573603826
loop 81, liks 18.639048573603826
loop 82, liks 18.639048573603826
loop 83, liks 18.639048573603826
loop 84, liks 18.639048573603826
loop 85, liks 18.639048573603826
loop 86, liks 18.639048573603826
loop 87, liks 18.639048573603826
loop 88, liks 18.639048573603826
loop 89, liks 18.639048573603826
loop 90, liks 18.639048573603826
loop 91, liks 18.639048573603826
loop 92, liks 18.639048573603826
loop 93, liks 18.639048573603826
loop 94, liks 18.639048573603826
loop 95, liks 18.639048573603826
loop 96, liks 18.639048573603826
loop 97, liks 18.639048573603826
loop 98, liks 18.639048573603826
loop 99, liks 18.639048573603826
Running fixm movers
loop 0, liks 11.032426369534418
loop 1, liks 18.500560742679443
loop 2, liks 18.634920051035103
loop 3, liks 18.638782723861468
loop 4, liks 18.639026770573672
loop 5, liks 18.639046485448514
loop 6, liks 18.6390483569000

# Finally, we can investigate the results

In [ ]:
BLM.plot_A1()